In [1]:
import numpy as np
from scipy.stats import norm
import datetime as dt
from scipy.integrate import quad
from math import pi
from math import log

In [7]:
S=846.9
r=0.40536/100  #From interpolation
today=dt.date(2013, 8, 30)
expiry=dt.date(2015, 1, 17)
T=(expiry-today).days/365.0
F=S*np.exp(r*T)
sigma=0.2581 #from part 2

In [12]:
def Black76_Lognormal_call(F,K,sigma,T):
    d1=(np.log(F/K)+((sigma**2)*T)/2)/(sigma*np.sqrt(T))
    d2=d1-sigma*np.sqrt(T)
    return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))
def Black76_Lognormal_put(F,K,sigma,T):
    return Black76_Lognormal_call(F,K,sigma,T)+np.exp(-r*T)*(K-F)
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return  K*np.exp(-r*T)*norm.cdf(-d2)-S*norm.cdf(-d1) 

def SABR(F, K, T, alpha= 0.99077728, beta=0.8, rho=-0.28514716, nu=0.35222713): 
    X = K
    if F == K:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

def payofffunction(S):
    h=S**3+2.5*np.log(S)+10
    return h


Payoff

In [13]:
#BS
#ST=S*np.exp((r-sigma**2)*T)

E1=(S*np.exp((r+sigma**2)*T))**3
E2=2.5*(log(S)+(r-0.5*sigma**2)*T)
V_BS=np.exp(-r*T)*(E1+E2+10)
V_BS

809935460.6267606

In [14]:
#Bachilier
#ST=S(1+sigma*Wt)
E1=(1+3*(sigma**2)*T)*(S**3)
E2=2.5*log(S)
x_lowbound=-1/(sigma*T**0.5)
E3=quad(lambda x: 2.5/np.sqrt(2*pi)*np.log(1+sigma*(T**0.5)*x)*np.exp(-0.5*(x**2)), x_lowbound, 5000)[0]
V_Bachilier=E1+E2+E3
V_Bachilier

775384948.0489168

Static Replication 

In [11]:
def callintegrand(K, S, r, T):
    sigma= SABR(F, K, T, alpha, beta, rho, nu)
    return 6*K*(BlackScholesCall(S, K, r, sigma, T))-2.5*((BlackScholesCall(S, K, r, sigma, T)/(K**2)))

def putintegrand(K, S, r, T):
    sigma= SABR(F, K, T, alpha, beta, rho, nu)
    return 6*K*(BlackScholesPut(S, K, r, sigma, T))-2.5*((BlackScholesPut(S, K, r, sigma, T)/(K**2)))

def SR(S,r,T):
    I_call = quad(lambda x: callintegrand(x, S, r, T), F, 5000000)[0]
    I_put  = quad(lambda x: putintegrand(x, S, r, T), 0.0, F)[0]
    Payoff=np.exp(-r*T)*payofffunction(F)
    return I_call+I_put+Payoff

print('Static Replication Price is',SR(S,r,T))

NameError: name 'alpha' is not defined

Varicance swap

In [156]:
#Black Scholes
def callintegrand1(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price


def putintegrand1(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

I_put = quad(lambda x: putintegrand1(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand1(x, S, r, T, sigma), F, 5000)
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('The expected integrated variance is: %.9f' % E_var)

#Check by definition
a=sigma**2*T
print('The Cross-check variance is: %.9f'%a)

The expected integrated variance is: 0.092166803
The Cross-check variance is: 0.092166803


In [158]:
#Bachilier
def BACall(S, K, sigma, T):
    d=(S-K)/(S*sigma*np.sqrt(T))
    return (S-np.exp(-r*T)*K)*norm.cdf(d) + (S*sigma*np.sqrt(T))*norm.pdf(d)

def BAPut(S, K, sigma, T):
    d=(S-K)/(S*sigma*np.sqrt(T))
    return (np.exp(-r*T)*K-S)*norm.cdf(-d)+S**sigma*np.sqrt(T)*norm.pdf(-d)
def callintegrand2(S, K, sigma, T):
    price = BACall(S, K, sigma, T) / K**2
    return price

def putintegrand2(S, K, sigma, T):
    price = BAPut(S, K, sigma, T) / K**2
    return price

I_put = quad(lambda x: putintegrand2(S, x, sigma, T), 0.0, F)
I_call = quad(lambda x: callintegrand2(S, x, sigma, T), F, 5000)
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('The expected integrated variance is: %.9f' % E_var)

#Check by definition
a=sigma**2*T
print('The Cross-check variance is: %.9f'%a)

The expected integrated variance is: -0.270236340
The Cross-check variance is: 0.092166803


/Users/jeowcj/anaconda3/lib/python3.6/site-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


In [161]:
#Static replication
def callintegrand3(K, S, r, T):
    sigma = SABR(F, K, T, alpha, beta, rho, nu)
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price


def putintegrand3(K, S, r, T):
    sigma= SABR(F, K, T, alpha, beta, rho, nu)
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

I_put = quad(lambda x: putintegrand3(x, S, r, T), 0.0, F)
I_call = quad(lambda x: callintegrand3(x, S, r, T), F, 10000)
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('Static Replication variance is: %.9f' % E_var)

Static Replication variance is: 0.104227450
